
# Interactive Map using Folium

This notebook generates an interactive map to visualize SpaceX launch sites and outcomes.

## Objectives:
- Plot global launch site locations.
- Visualize launch outcome using colored markers.
- Show proximities (railways, coastlines, etc.) from selected launch site.


## Import Required Libraries

In [20]:
import pandas as pd
import folium
from folium import plugins


## Step 1: Create a mapping of known SpaceX launch sites to their coordinates

In [21]:
# Load your cleaned dataset
df = pd.read_csv("spacex_clean_data.csv")

# Define a dictionary with known SpaceX launch site coordinates
site_coords = {
    'Cape Canaveral, SLC‑40': (28.5618571, -80.577366),
    'Kennedy, LC‑39A': (28.6080585, -80.6039558),
    'Vandenberg, SLC‑4E': (34.6328344, -120.6107465),
    'Vandenberg, SLC‑4W': (34.633, -120.61),  # Historic pad
    'Vandenberg, SLC-6': (34.576, -120.629),
    'Cape Canaveral, LC-13': (28.485833, -80.544444),
    'Cape Canaveral, LC-36A': (28.508, -80.55),
    'Boca Chica': (25.997, -97.156)
}


## Step 2: Map the coordinates into your DataFrame

In [22]:
# Map latitude and longitude from launch_site column
df['latitude'] = df['launch_site'].map(lambda x: site_coords.get(x, (None, None))[0])
df['longitude'] = df['launch_site'].map(lambda x: site_coords.get(x, (None, None))[1])

# Drop rows with unknown coordinates (if any)
df = df.dropna(subset=['latitude', 'longitude'])

# Preview updated DataFrame
df[['launch_site', 'latitude', 'longitude']].drop_duplicates()
df.to_csv("spacex_clean_data_with_coordinates.csv", index=False)
print("✅ Updated dataset saved with coordinates!")
df.head()


✅ Updated dataset saved with coordinates!


,flight_no.,date_and_time_(utc),"version,_booster[h]",launch_site,payload[i],payload_mass,orbit,customer,launch_outcome,booster_landing,latitude,longitude
0,195,"January 3, 2023 14:56[17]",F9 B5 B1060‑15,"Cape Canaveral, SLC‑40",Transporter-6 (115 payload smallsat rideshare),Unknown[j],SSO,Various,Success,Success (LZ‑1),28.561857,-80.577366
2,196,"January 10, 2023 04:50[23]",F9 B5 B1076‑2,"Cape Canaveral, SLC‑40",OneWeb 16 (40 satellites),"6,000 kg (13,000 lb)",Polar LEO,OneWeb,Success,Success (LZ‑1),28.561857,-80.577366
4,FH 5,"January 15, 2023 22:56[29]",Falcon Heavy B5 B1070 (core),"Kennedy, LC‑39A",USSF-67 (CBAS-2 & LDPE-3A),"~3,750 kg (8,270 lb)",GEO,USSF,Success,No attempt,28.608058,-80.603956
5,FH 5,"January 15, 2023 22:56[29]",B1064‑2 (side),"Kennedy, LC‑39A",USSF-67 (CBAS-2 & LDPE-3A),"~3,750 kg (8,270 lb)",GEO,USSF,Success,Success (LZ‑2),28.608058,-80.603956
6,FH 5,"January 15, 2023 22:56[29]",B1065‑2 (side),"Kennedy, LC‑39A",USSF-67 (CBAS-2 & LDPE-3A),"~3,750 kg (8,270 lb)",GEO,USSF,Success,Success (LZ‑1),28.608058,-80.603956


## Map 1: All Launch Sites with Location Markers

In [23]:
# Map 1: All Launch Sites
site_map = folium.Map(location=[28.563197, -80.576820], zoom_start=3)

for _, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['launch_site'],
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(site_map)

site_map.save("folium_launch_sites_map.html")
site_map


## Map 2: Launch Outcomes with Colored Markers

In [24]:
# Map 2: Outcome markers with color
outcome_map = folium.Map(location=[28.563197, -80.576820], zoom_start=3)

def color(success):
    if "Success" in str(success):
        return 'green'
    elif "Failure" in str(success):
        return 'red'
    else:
        return 'gray'

for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=6,
        color=color(row.get('booster_landing', 'None')),
        fill=True,
        fill_opacity=0.7,
        popup=row.get('booster_landing', 'N/A')
    ).add_to(outcome_map)

outcome_map.save("folium_outcome_map.html")
outcome_map


## Map 3: Proximity of Launch Site to Key Features

In [25]:
# Map 3: Proximity map (static example)
selected = df[df['launch_site'].str.contains("Cape Canaveral", na=False)].iloc[0]

prox_map = folium.Map(location=[selected['latitude'], selected['longitude']], zoom_start=10)

# Launch Site marker
folium.Marker([selected['latitude'], selected['longitude']], popup="Launch Site").add_to(prox_map)

# 5 km Circle around launch site
folium.Circle([selected['latitude'], selected['longitude']], radius=5000, color='blue', fill=False).add_to(prox_map)

# Simulated Nearby Highway
folium.Marker([selected['latitude'] + 0.03, selected['longitude']], popup="Nearby Highway").add_to(prox_map)

# Simulated Nearby Coastline
folium.Marker([selected['latitude'], selected['longitude'] + 0.03], popup="Nearby Coastline").add_to(prox_map)

prox_map.save("folium_proximity_map.html")
prox_map
